In [ ]:
pip install neo4j langchain openai pandas scikit-learn matplotlib seaborn


In [ ]:
pip install sentence-transformers

In [ ]:
pip install -U langchain-openai

In [ ]:
pip install faiss-cpu

In [ ]:
pip install -U langchain-openai


In [ ]:
!pip install faiss-cpu

In [ ]:
pip install -U langchain-community

#RAG (SOUP Data)

In [ ]:
# STEP 1: Setup dependencies
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from sklearn.metrics import classification_report
import openai
import json
import os

In [ ]:

# === OpenAI Key Setup ===
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


In [ ]:
# STEP 2: Load train.json into documents for indexing
with open("train.json") as f:
    train_data = json.load(f)

documents = []
for item in train_data:
    content = f"Comment: {item['comment']}\nBefore: {item['code_before']}\nAfter: {item['code_after']}\nLabel: {'Valid' if item['valid'] == 1 else 'Invalid'}"
    documents.append(Document(page_content=content))


In [ ]:
import openai

client = openai.OpenAI(api_key="")



In [ ]:
# STEP 3: Load training data and create documents
with open("") as f:
    train_data = json.load(f)

In [ ]:
documents = []
for item in train_data:
    content = f"Comment: {item['comment']}\nBefore: {item['code_before']}\nAfter: {item['code_after']}\nLabel: {'Valid' if item['valid'] == 1 else 'Invalid'}"
    documents.append(Document(page_content=content))


In [ ]:
# STEP 4: Build FAISS vectorstore
embedding = OpenAIEmbeddings(openai_api_key="")  # same key here
vectorstore = FAISS.from_documents(documents, embedding)


In [ ]:

# STEP 5: Define retrieval function
retriever = vectorstore.as_retriever(search_type="similarity", k=3)

def get_similar_examples(comment):
    results = retriever.get_relevant_documents(comment)
    return results

def format_context(docs):
    return "\n\n".join([doc.page_content for doc in docs])


In [ ]:
# STEP 6: Define prediction function
def predict(comment, code_before, code_after):
    examples = get_similar_examples(comment)
    context = format_context(examples)

    prompt = f"""
You are a Stack Overflow moderator. You are given a post-edit scenario:
- A comment made on Stack Overflow: {comment}
- The code before edit: {code_before}
- The code after edit: {code_after}

Some previous related comment-edit pairs:
{context}

Decide if the comment justifies the code edit. Answer with "Valid" or "Invalid" only.
""".strip()

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

In [ ]:
# === STEP 7: Evaluate on test.json ===
with open("test.json") as f:
    test_data = json.load(f)

y_true, y_pred = [], []

for item in test_data:
    gt = "Valid" if item["valid"] == 1 else "Invalid"
    pred = predict(item["comment"], item["code_before"], item["code_after"])

    y_true.append(gt.lower())
    y_pred.append(pred.lower())
    print(f"GT: {gt}, PRED: {pred}")
    print(f"Comment: {item['comment']}")
    print(f"Code before: {item['code_before']}")
    print(f"Code after: {item['code_after']}")
    print(f"Examples: {get_similar_examples(item['comment'])}")
    print("\n")

print(classification_report(y_true, y_pred))

GT: Invalid, PRED: Valid
Comment: But that´s the problem. I need all the numbers who are from an interval . For example: i have the nubmbers from  100 to 1700 and the progrogramm should outpout all the armstrong numbers from 100 to 1700 but where it´s going, the programm only outputs armstrong numbers till 1000 (r*r*r) or all armstrong numbers higher then 1000(r*r*r*r). And I want them to be all togheter and not seperate.
Code before: int tmp = r; for (int i = 0; i < String.valueOf(number).length(); i++) { tmp = tmp * r; } sum = sum + tmp;
Code after: int tmp = 1; for (int i = 0; i < String.valueOf(number).length(); i++) { tmp = tmp * r; } sum = sum + tmp;
Examples: [Document(id='c6130cd4-d008-40db-8469-c06a42c94d94', metadata={}, page_content='Comment: It worked but I also need disvible by 5 it\'s high and if it\'s divisible by 3 and 5 it\'s tigh. I tried myself but It would repeat the numbers.\nBefore: int count; for(count = 1; count <= 99;count++){ if(count%3==0 ){ System.out.print(

In [ ]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

     invalid       0.92      0.69      0.79       373
       valid       0.47      0.83      0.60       127

    accuracy                           0.72       500
   macro avg       0.70      0.76      0.69       500
weighted avg       0.81      0.72      0.74       500



In [ ]:
import pandas as pd

# Load the JSON file into a DataFrame
# Replace 'your_file.json' with the actual path to your JSON file
try:
    df = pd.read_csv('/content/rag_responses_formatted.csv')
    print("DataFrame created successfully:")
    display(df.head())
except FileNotFoundError:
    print("Error: 'your_file.json' not found. Please update the path to your JSON file.")
except Exception as e:
    print(f"An error occurred: {e}")

DataFrame created successfully:


before  \
0  int tmp = r; for (int i = 0; i < String.valueO...   
1  @Entity public class Address{ @Id @Column("add...   
2  public boolean isStringValid(String s){ return...   
3  String x = "NewString"; int y = 5; double z = ...   
4  public static String readFileAsString(File fil...   

                                               after  \
0  int tmp = 1; for (int i = 0; i < String.valueO...   
1  @Entity public class Address{ @Id @Column("add...   
2  public boolean isStringValid(String s){ return...   
3  String x = "NewString"; int y = 5; double z = ...   
4  public static String readFileAsString(File fil...   

                                             comment rag_predict  \
0  But that´s the problem. I need all the numbers...       Valid   
1  yes you are correct. but some one already desi...     Invalid   
2  @Tom: wrong regex. "[^abc]+" means a sequence ...     Invalid   
3  I think that still falls under the category of...     Invalid   
4  Thanks all for your immense help. I learnt a g...     Invalid   

   response_time_min gt_label  
0                 43  Invalid  
1                  2  Invalid  
2               1977  Invalid  
3                  1  Invalid  
4                 52  Invalid